# Imports

In [ ]:
!pip install transformers
# # !pip install torch==1.5.0 
# !pip install torchvision==0.4.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import pickle
import random
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import os
from enum import Enum
from torch.nn import functional as F
import time
import logging
import numpy as np


from transformers import DistilBertTokenizerFast, DistilBertModel

logger = logging.getLogger(__name__)
random.seed(13)

# Utilities

In [ ]:
def pytorch_cos_sim(a: torch.as_tensor, b: torch.as_tensor):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    This function can be used as a faster replacement for 1-scipy.spatial.distance.cdist(a,b)
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.as_tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.as_tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

# Dataset and Dataloader

### Config

In [ ]:
folder_quora = '/gdrive/MyDrive/quora_android_2'
folder = '/gdrive/MyDrive/MultiModal'
img_dir = '/gdrive/MyDrive/MultiModal/data'
BATCH_SIZE = 1
fin_BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


###Loading Data

In [ ]:
with open(folder_quora+'/data/splits/pandas_split.txt','rb') as a:
    train_qr=pickle.load(a)
    dev_qr = pickle.load(a)
    test_qr = pickle.load(a)

In [ ]:
with open(folder+'/data/splits/data_pos_neg.txt','rb') as a:
    train_data_pos = pickle.load(a)
    train_data_neg = pickle.load(a)
    dev_data_pos = pickle.load(a)
    dev_data_neg = pickle.load(a)
    test_data_pos = pickle.load(a)
    test_data_neg = pickle.load(a)

In [ ]:
with open('/gdrive/MyDrive/sbert_supervised(for_data_only)/android_nosw_rearranged.txt','rb') as a:
    score_qr = pickle.load(a)

### Creating Dataset and Dataloader class 

In [ ]:
# !pip install sentence-transformers

In [ ]:
# from sentence_transformers import models, losses, util, SentenceTransformer, SentencesDataset, InputExample, evaluation
# from sentence_transformers.cross_encoder import CrossEncoder

In [ ]:
from PIL import Image
from torchvision import transforms
transform_pipe = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
class custom_dataset(Dataset):
    def __init__(self,qr,qr_idx,img_dir,label,transform = None,score_qr = score_qr):
        self.qr = qr
        self.qr_idx = qr_idx
        self.img_dir = img_dir
        self.label = label
        self.transform = transform
        self.score_qr = score_qr

    def image_adder(self,id1):
        img_id1 = list()
        if((self.qr.at[id1,'Attachments'])!=None):
            for i in self.qr.at[id1,'Attachments']:
                try:
                    img_path = os.path.join(self.img_dir,i)
                    img = Image.open(img_path).convert('RGB')
                    if(self.transform):
                        img = self.transform(img)
                        img.reshape(3,224,224)
                    img_id1.append(img)
                except Exception as e: 
                    print(e)
        else:
            img_id1.append(torch.zeros(3,224,224))

        # Work on this, for few examples, it is still saying list index out of range
        if(len(img_id1)==0):
            # print('No attachments found for id {}'.format(id1))
            print(f'Something went wrong with the image of id {id1}')
            img_id1.append(torch.zeros(3,224,224))

        return img_id1
    
    def __getitem__(self,idx):
        id1 = self.qr_idx[idx][0]
        id2 = self.qr_idx[idx][1]
        img_id1 = self.image_adder(id1)
        img_id2 = self.image_adder(id2)

        # print(len(img_id1))

        # print('Printing id1 {} and len {} and id2 {} and len {} '.format(
        #     id1,len(img_id1),
        #     id2, len(img_id2)
        # ))

        # print('Printing id1 shape {} and id2  shape {}'.format(
        #     img_id1[0].shape,
        #     img_id2[0].shape
        # ))        

        sample = {
            'image': [img_id1[0],img_id2[0]]    #Currently taking only one input image
        }

        t1 = '[CLS]' + self.qr.loc[id1,'Title'] + ' ' + ' '.join(self.qr.loc[id1,'Tags']) + ' ' + self.qr.loc[id1,'Text'] + '[SEP]'
        t2 = '[CLS]' + self.qr.loc[id2,'Title']+ ' ' + ' '.join(self.qr.loc[id2,'Tags']) + ' ' + self.qr.loc[id2,'Text'] + '[SEP]'
        tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
        indexed_t1 = tokenizer.encode(t1,max_length = 512,truncation = True)
        indexed_t2 = tokenizer.encode(t2,max_length = 512,truncation = True)
        # t2_token = tokenizer.tokenize(t2)
        # indexed_t1 = tokenizer.convert_tokens_to_ids(t1_token)
        # indexed_t2 = tokenizer.convert_tokens_to_ids(t2_token)

        while(len(indexed_t1)<512):
            indexed_t1.append(0)
        while(len(indexed_t2)<512):
            indexed_t2.append(0)

        ten_t1 = torch.as_tensor(indexed_t1)[:512]
        ten_t2 = torch.as_tensor(indexed_t2)[:512]
        
        # related_score = list()
        for l in self.label:
            try:
                if(l==torch.tensor(1)):
                    related_score=(score_qr.loc[id1,'Related_Scores'][score_qr.loc[id1,'New_Related'].index(id2)])
                else:
                    related_score=(torch.tensor(0.0))
            except:
                related_score = torch.tensor(0.0)

        try:
            sample["label"] = self.label[idx]
            sample["token"] = [ten_t1,ten_t2] # torch.Size([batch_size, 512])
            sample['score'] = related_score
        except Exception as e:
            print(e)
        
        return sample

    def __len__(self):
        return len(self.label)

### Intializing dataset and dataloader

In [ ]:
train_dataset = custom_dataset(train_qr,train_data_pos + train_data_neg,
                               img_dir,torch.ones(len(train_data_pos))+
                               torch.zeros(len(train_data_neg)),
                               transform = transform_pipe)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = 8,
    shuffle = True
    )

In [ ]:
dev_dataset = custom_dataset(pd.concat([train_qr,dev_qr]),dev_data_pos + dev_data_neg,
                               img_dir,torch.ones(len(dev_data_pos))+
                               torch.zeros(len(dev_data_neg)),
                               transform = transform_pipe)

In [ ]:
dev_loader = DataLoader(
    dev_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = 8,
    shuffle = True
    )

In [ ]:
test_dataset = custom_dataset(pd.concat([train_qr,test_qr]),test_data_pos + test_data_neg,
                               img_dir,torch.ones(len(test_data_pos))+
                               torch.zeros(len(test_data_neg)),
                               transform = transform_pipe)

In [ ]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = 8,
    shuffle = True
    )

#Model

### Downloading models

In [ ]:
from torchvision import models
res50 = models.resnet50(pretrained = True)
res50 = res50.to(device)
for param in res50.parameters():
    param.requires_grad = True
# res50.eval()

distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
distilbert = distilbert.to(device)
for param in distilbert.parameters():
    param.requires_grad = True


### Forward Hook

In [ ]:
def get_activation(name, activation):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

### 2d intra attention layer

In [ ]:
import math
class Final2d(torch.nn.Module):
    """ Custom Linear layer for equation (9) in paper --> 2D intra attention layer""" 
    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        weights = torch.Tensor(size_out, size_in)
        self.weights = torch.nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.

        # initialize weights and biases
        torch.nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(self.weights)

    def forward(self, x, a):
        # x.shape = batch_size, 768, 119
        # a.shape = batch_size, 119
        n = x.shape[2]
        h_cap = torch.zeros(x.shape[0], x.shape[1], device = device)
        # for k in range(x.shape[0]):
        #     for i in range(n):
        #         h_cap[k] = h_cap[k] + a[k][i]*torch.matmul(self.weights, x[k, :,i])
        z = torch.matmul(self.weights, x)
        for k in range(x.shape[0]):
            h_cap[k] = torch.sum(a[k, :]*z[k, :, :], dim = 1)
        return h_cap

### Mask

In [ ]:
n_val = 360+49 #360 is the text input vector length, and 49 is the image vector length
sz = (BATCH_SIZE, n_val*n_val, 1)
mask = torch.zeros(sz, dtype=torch.bool, device = device)
for i in range(n_val):
    mask[:, i*n_val+i] = 1
        
for i in range(n_val-50, n_val):
    for j in range(n_val-50, n_val):
        mask[:, i*n_val+j] = 1

### Model class

In [ ]:
class BridgeModel(torch.nn.Module):
    # define model elements
    def __init__(self):
        super(BridgeModel, self).__init__()
        # input to first hidden layer
        self.activation = {}
        # self.res50 = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
        # self.res50 = models.resnet50(pretrained = True)
        # res50.eval()
        self.res50 = res50.to(device)
        self.res50.layer4[2].relu.register_forward_hook(get_activation('conv1', self.activation))
        self.res50.avgpool.register_forward_hook(get_activation('conv2', self.activation))
        self.bridge_seven_conv = torch.nn.Conv2d(in_channels=2048, out_channels=768, kernel_size=1, stride=1, padding = 0)
        self.bridge_one_conv = torch.nn.Conv2d(in_channels=2048, out_channels=768, kernel_size=1, stride=1, padding = 0)
        self.flatten = torch.nn.Flatten(start_dim = 2)
        self.distilbert = distilbert.to(device)
        # self.mod_v = list()
        self.mod_v = 360 #Overall text token length
        self.linear_2d = torch.nn.Linear(768*2, 1)
        self.soft_2d = torch.nn.Softmax(dim = 1)
        self.final_2d = Final2d(768, 768) #2D intra attention layer
        self.fc = torch.nn.Linear(768+2048+768, 500)
        # self.sigm = torch.nn.Sigmoid()
    # forward propagate input
    def forward(self, X1, X2):
        # input to first hidden layer/
        res_out = self.res50(X1)  # torch.Size([batch_size, 1000])
        seven_conv = self.activation['conv1'] # torch.Size([batch_size, 2048, 7, 7])
        one_conv = self.activation['conv2'] # torch.Size([batch_size, 2048, 1, 1])
        out3 = one_conv # torch.Size([batch_size, 2048, 1, 1])
        out3 = out3[:, :, 0, 0].clone()
        seven_conv = self.bridge_seven_conv(seven_conv)  # torch.Size([batch_size, 768, 7, 7])
        one_conv = self.bridge_one_conv(one_conv)  # torch.Size([batch_size, 768, 1, 1])
        seven_conv = self.flatten(seven_conv) # torch.Size([batch_size, 768, 49])
        one_conv = self.flatten(one_conv) # torch.Size([batch_size, 768, 1])
        bridge_cat_temp = torch.cat((one_conv, seven_conv), dim = 2) # torch.Size([1, 768, 50])
        bridge_cat = bridge_cat_temp.permute(0, 2, 1) # torch.Size([1, 50, 768])
        distilbert_embed = self.distilbert.embeddings(input_ids = X2) # torch.Size([1, 512, 768])
        distilbert_embed = distilbert_embed[:,:self.mod_v,:]
        attn_mask = torch.ones(distilbert_embed.shape[0],self.mod_v + 50).to(device)
        prev = torch.cat((distilbert_embed, bridge_cat), dim = 1) #562
        prev2 = prev[:, self.mod_v:, :]
        
        for i in range(6):
            part1 = self.distilbert.transformer.layer[i](prev,attn_mask)[0] # torch.Size([1, 562, 768])
            part2 = torch.matmul(part1[:, self.mod_v:, :], self.distilbert.transformer.layer[i].attention.v_lin.weight)
            part1 = part1[:, :self.mod_v, :] # torch.Size([1, 512, 768])
            part1 = torch.cat((part1, part2), dim = 1)
            prev = part1
            prev2 = prev[:, self.mod_v:, :]
            
        out1 = prev[:, 0, :]
        prev = prev[:, 1:, :]
        
        n = self.mod_v + 49 # length(n) = |v| + |g|

        # mask = self.mask_creater(BATCH_SIZE,n) 

        hij = torch.zeros(prev.shape[0], n*n, 2*prev.shape[2], device = device) # batch_size, 119*119=14161, 768+768
        # for k in range(prev.shape[0]):
        #    for i in range(n):
        #        for j in range(n):
        #            hij[k][i*n+j] = torch.cat((prev[k][i], prev[k][j]), dim = 0)

        prev_repeat1 =  prev.repeat(1, n, 1) 
        prev_repeat2 = torch.repeat_interleave(prev, repeats = n, dim = 1)
        hij = torch.cat((prev_repeat1, prev_repeat2), dim = 2)

        sij = self.linear_2d(hij) # batch_size, 14161, 1
        
        # aij = self.soft_2d(sij) # batch_size, 14161, 1
        
        sij[mask] = -1000

        aij_tmp = self.soft_2d(sij) # batch_size, 14161, 1

        aij = aij_tmp.reshape(aij_tmp.shape[0], n, n) # batch_size, n, n
        
        ai_cap = torch.zeros(prev.shape[0], n, device = device) # batch_size, n
        
        ai_cap = (torch.sum(aij, dim = 1) + torch.sum(aij, dim = 2))/2
        
        #for k in range(prev.shape[0]):
         #   for i in range(n):
          #      for j in range(n):
           #         ai_cap[k][i] = ai_cap[k][i] + aij[k][i*n+j][0]/2
            #        ai_cap[k][i] = ai_cap[k][i] + aij[k][j*n+i][0]/2
        
        new_hi = prev.permute(0, 2, 1)
        out2 = self.final_2d(new_hi, ai_cap) # torch.Size([batch_size, 768])
        
        out = torch.cat((out1, out2, out3), dim = 1) # torch.Size([batch_size, 2048+768+768])
        out = self.fc(out) # torch.Size([batch_size, 500])
        # out = torch.nn.functional.normalize(out, p=2, dim=1, eps=1e-12) #L2 normalizing the final tensor
        
        return out

    # def mask_creater(self,n_val,BATCH_SIZE=BATCH_SIZE):
    #     print(n_val)
    #     sz = (BATCH_SIZE, n_val*n_val, 1)
    #     mask = torch.zeros(sz, dtype=torch.bool, device = device)
    #     for i in range(n_val):
    #         mask[:, i*n_val+i] = 1
                
    #     for i in range(n_val-50, n_val):
    #         for j in range(n_val-1, n_val):
    #             mask[:, i*n_val+j] = 1
        
    #     return mask

# Losses

## Multiple Negative Ranking loss

In [ ]:
class negrankloss(nn.Module):
    def __init__(self,scale: float = 20.0,reduction:str = 'mean'):
        # self.cosine_sim = nn.CosineSimilarity()
        super().__init__()
        self.red = reduction
        self.scale = scale
        self.cross_entropy = nn.CrossEntropyLoss(reduction=self.red)

    def cal_loss(self,emb1: torch.as_tensor,emb2: torch.as_tensor, label: torch.Tensor):
        scores  = pytorch_cos_sim(emb1,emb2) 
        # print(f'The scores of cosine similarity for MNRloss is {scores}')
        labels = torch.as_tensor(range(len(scores)), dtype=torch.long, device=scores.device)
        loss = self.cross_entropy(scores, labels)
        return loss

In [ ]:
MNRloss = negrankloss()

## Online Constrantive Loss


In [ ]:
class SiameseDistanceMetric(Enum):
    """
    The metric for the contrastive loss
    """
    EUCLIDEAN = lambda x, y: F.pairwise_distance(x, y, p=2)
    MANHATTAN = lambda x, y: F.pairwise_distance(x, y, p=1)
    COSINE_DISTANCE = lambda x, y: 1-F.cosine_similarity(x, y)


In [ ]:
class OnlineConstrantiveLoss(nn.Module):
    def __init__(self,distance_metric=SiameseDistanceMetric.COSINE_DISTANCE,margin: float = 0.5):
        super().__init__()
        self.distance_metric = distance_metric
        self.margin = margin

    def cal_loss(self,emb1,emb2,labels,size_average=False):
        distance_matrix = self.distance_metric(emb1,emb2)
        # print(f'distance matrix of OCloss is {distance_matrix}')
        negs = distance_matrix[labels == 0]
        poss = distance_matrix[labels == 1]
        # print(f'positive and negatives are {poss} and {negs}')

        # select hard positive and hard negative pairs
        # But for current batch size of 1, it is impossible to consider hard positive and hard negative
        # negative_pairs = negs[negs < (poss.max() if len(poss) > 1 else negs.mean())]
        # positive_pairs = poss[poss > (negs.min() if len(negs) > 1 else poss.mean())]

        negative_pairs = negs
        positive_pairs = poss

        # print(f'positive and negative pairs are {positive_pairs} and {negative_pairs}')

        positive_loss = positive_pairs.pow(2).sum()
        negative_loss = F.relu(self.margin - negative_pairs).pow(2).sum()
        # print(f'positive loss is {positive_loss} and negative loss is {negative_loss}')
        loss = positive_loss + negative_loss
        return loss
    

In [ ]:
OCloss = OnlineConstrantiveLoss()

## Cosine Similarity Loss

In [ ]:
class CosineSimilarityLoss(nn.Module):
    def __init__(self, loss_fct = nn.MSELoss(), cos_score_transformation=nn.Identity()):
        super(CosineSimilarityLoss, self).__init__()
        self.loss_fct = loss_fct
        self.cos_score_transformation = cos_score_transformation


    def cal_loss(self,emb1:torch.Tensor,emb2:torch.Tensor, labels: torch.Tensor):
        output = self.cos_score_transformation(torch.cosine_similarity(emb1,emb2))
        return self.loss_fct(output, labels.view(-1))

In [ ]:
Cosloss = CosineSimilarityLoss()

# Calculating loss and optimizing

In [ ]:
from torch.optim import Adam
from torch.nn import BCELoss
import time
import copy
from transformers import get_linear_schedule_with_warmup

def train_model(train_loader, model,num_epochs):
    model.train()

    since = time.time()

    optimizer = Adam(model.parameters())
    train_steps = num_epochs

    acc_steps = fin_BATCH_SIZE/BATCH_SIZE

    use_amp = False
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    
    # enumerate epochs
    for epoch in range(num_epochs):

        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        running_OCloss = 0.0
        running_MNRloss = 0.0
        running_Cosloss = 0.0

        for i, batch in enumerate(train_loader):

            with torch.cuda.amp.autocast(enabled=use_amp):   
                images = batch['image']
                token = batch['token']
                label = batch['label']
                score = batch['score']

                images[0],images[1] = images[0].to(device),images[1].to(device)
                token[0],token[1] = token[0].to(device), token[1].to(device)
                label = label.to(device)
                score = score.to(device)
                
                # compute the model output
                yhat1 = model(images[0],token[0])
                yhat2 = model(images[1],token[1])

                OCloss_val = OCloss.cal_loss(yhat1,yhat2,label)
                running_OCloss +=  OCloss_val.item()*BATCH_SIZE
                OCloss_val = OCloss_val/acc_steps
                # print(f'Ocloss values is {OCloss_val.item()}')
    
                MNRloss_val = MNRloss.cal_loss(yhat1,yhat2,label)
                running_MNRloss += MNRloss_val.item()*BATCH_SIZE
                MNRloss_val = MNRloss_val/acc_steps
                 # print(f'MNRloss values is {MNRloss_val}')

                Cosloss_val = Cosloss.cal_loss(yhat1,yhat2,score)
                running_Cosloss += Cosloss_val.item()*BATCH_SIZE
                Cosloss_val = Cosloss_val/acc_steps


            scaler.scale(OCloss_val).backward(retain_graph = True)
            scaler.scale(MNRloss_val).backward(retain_graph=True)
            scaler.scale(Cosloss_val).backward()

            if((i+1)%acc_steps == 0):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            if(i%30 == 0 ):
                print(' OCloss is {} and MNRloss is {} and Cosloss is {} and time taken is {} after {} iterations'.format(
                    running_OCloss/((i+1)*BATCH_SIZE),
                    running_MNRloss/((i+1)*BATCH_SIZE),
                    running_Cosloss/((i+1)*BATCH_SIZE),
                    time.time()-since,
                    i))

            del yhat1,yhat2, images, label, token, score, OCloss_val, Cosloss_val, MNRloss_val

        save_dir = folder + '/model_state_dict'
        os.makedirs(save_dir, exist_ok=True) 
        torch.save(model.state_dict(), save_dir + '/distilbert_cossim.bin') 
        print(f"Model Saved after epoch {epoch}")

# Training the model

In [ ]:
torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

In [ ]:
!nvidia-smi

Mon Feb 15 18:11:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    30W /  70W |   1328MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model = BridgeModel().to(device)
print(sum(p.numel() for p in model.parameters()))

97451037


In [ ]:
train_model(train_loader,model,1)

Epoch 0/0
----------
 OCloss is 0.002375280950218439 and MNRloss is 0.6944969296455383 and Cosloss is 0.18439336121082306 and time taken is 42.082067012786865 after 0 iterations
 OCloss is 0.03494626876944551 and MNRloss is 0.7028353329627745 and Cosloss is 0.25314821399027304 and time taken is 180.1316978931427 after 30 iterations
 OCloss is 0.01776020457117371 and MNRloss is 0.6980558262496698 and Cosloss is 0.2520949334883299 and time taken is 348.59856247901917 after 60 iterations
 OCloss is 0.011906538631371128 and MNRloss is 0.6964812173948183 and Cosloss is 0.24214543582318904 and time taken is 515.3554110527039 after 90 iterations
 OCloss is 0.00895527454179868 and MNRloss is 0.6956738164602232 and Cosloss is 0.24628667419483838 and time taken is 682.9870879650116 after 120 iterations
 OCloss is 0.007177171607264861 and MNRloss is 0.69517648614795 and Cosloss is 0.24278666071643104 and time taken is 815.4570307731628 after 150 iterations
 OCloss is 0.005990727226712252 and MNRl

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 OCloss is 0.0009252511661413488 and MNRloss is 0.6931818946759225 and Cosloss is 0.23302959788341243 and time taken is 17452.928606987 after 3240 iterations
 OCloss is 0.0013239819040468915 and MNRloss is 0.6932093867720613 and Cosloss is 0.2333045216366644 and time taken is 17609.810970067978 after 3270 iterations
 OCloss is 0.0013119691908857769 and MNRloss is 0.6932097517949312 and Cosloss is 0.2332696698727896 and time taken is 17752.01113176346 after 3300 iterations
 OCloss is 0.0013001601946285273 and MNRloss is 0.6932071743516799 and Cosloss is 0.23293365239403657 and time taken is 17910.309165477753 after 3330 iterations
 OCloss is 0.001288576554039919 and MNRloss is 0.6932069812101893 and Cosloss is 0.23320273558779048 and time taken is 18073.736450195312 after 3360 iterations
 OCloss is 0.0012771946008460495 and MNRloss is 0.6932072223234725 and Cosloss is 0.2331420189191004 and time taken is 18238.942098379135 after 3390 iterations
 OCloss is 0.001266010320667285 and MNRlos

In [ ]:
model.load_state_dict(torch.load(folder+'/model_state_dict/distilbert.bin'))

<All keys matched successfully>

# Evaluator

In [ ]:
model = BridgeModel().to(device)

In [ ]:
model.load_state_dict(torch.load('/gdrive/MyDrive/MultiModal/model_state_dict/distilbert.bin'))

<All keys matched successfully>

## Binary Classification Evaluator

### Imports

In [ ]:
# from . import SentenceEvaluator
import logging
import os
import csv
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from sklearn.metrics import average_precision_score
import numpy as np
from typing import List
# from ..readers import InputExample


logger = logging.getLogger(__name__)


### Functiion

In [ ]:
class BinaryClassificationEvaluator():
    """
    Evaluate a model based on the similarity of the embeddings by calculating the accuracy of identifying similar and
    dissimilar sentences.
    The metrics are the cosine similarity as well as euclidean and Manhattan distance
    The returned score is the accuracy with a specified metric.
    The results are written in a CSV. If a CSV already exists, then values are appended.
    The labels need to be 0 for dissimilar pairs and 1 for similar pairs.
    :param sentences1: The first column of sentences
    :param sentences2: The second column of sentences
    :param labels: labels[i] is the label for the pair (sentences1[i], sentences2[i]). Must be 0 or 1
    :param name: Name for the output
    :param batch_size: Batch size used to compute embeddings
    :param show_progress_bar: If true, prints a progress bar
    :param write_csv: Write results to a CSV file
    """

    def __init__(self,
                 dataset,
                 name: str = '',
                 batch_size: int = 32,
                 show_progress_bar: bool = False,
                 write_csv: bool = True
                 ):
        
        self.dataset = dataset
        self.labels = list()
        self.write_csv = write_csv
        self.name = name
        self.batch_size = batch_size
        self.dataloader = DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            pin_memory=True,
            num_workers = 8,
            shuffle = True
        )

        if show_progress_bar is None:
            show_progress_bar = (logger.getEffectiveLevel() == logging.INFO or logger.getEffectiveLevel() == logging.DEBUG)
        self.show_progress_bar = show_progress_bar

        self.csv_file = "binary_classification_evaluation" + ("_"+name if name else '') + "_results.csv"
        self.csv_headers = ["epoch", "steps",
                            "cosine_acc", "cosine_acc_threshold", "cosine_f1", "cosine_precision", "cosine_recall", "cosine_f1_threshold", "cosine_average_precision",
                            "manhatten_acc", "manhatten_acc_threshold", "manhatten_f1", "manhatten_precision", "manhatten_recall", "manhatten_f1_threshold", "manhatten_average_precision",
                            "eucledian_acc", "eucledian_acc_threshold", "eucledian_f1", "eucledian_precision", "eucledian_recall", "eucledian_f1_threshold", "eucledian_average_precision"]


    # @classmethod
    # def from_input_examples(cls, examples: List[InputExample], **kwargs):
    #     sentences1 = []
    #     sentences2 = []
    #     scores = []

    #     for example in examples:
    #         sentences1.append(example.texts[0])
    #         sentences2.append(example.texts[1])
    #         scores.append(example.label)
    #     return cls(sentences1, sentences2, scores, **kwargs)

    def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1) -> float:

        if epoch != -1:
            if steps == -1:
                out_txt = f" after epoch {epoch}:"
            else:
                out_txt = f" in epoch {epoch} after {steps} steps:"
        else:
            out_txt = ":"

        logger.info("Binary Accuracy Evaluation of the model on " + self.name + " dataset" + out_txt)
        
        embeddings1 = list()
        embeddings2 = list()

        with torch.no_grad():
            model.eval()
            for i, batch in enumerate(self.dataloader):
                images = batch["image"]
                label = batch["label"]
                label = label.float()
                token = batch["token"]
                
                # print(images,label,token)

                images[0],images[1] = images[0].to(device),images[1].to(device)
                token[0],token[1] = token[0].to(device), token[1].to(device)
                label = label.to(device)
                
                # compute the model output
                yhat1 = model(images[0], token[0])
                yhat2 = model(images[1],token[1])

                for j in yhat1:
                    embeddings1.append(j.cpu().detach().numpy())
                for j in yhat2:
                    embeddings2.append(j.cpu().detach().numpy())
                for j in label:
                    self.labels.append(float(j))
                
                if(i%30==0 and i!=0):
                    print(f'Completed {i} iterations')

        cosine_scores = 1-paired_cosine_distances(embeddings1[:400], embeddings2[:400])
        manhattan_distances = paired_manhattan_distances(embeddings1[:400], embeddings2[:400])
        euclidean_distances = paired_euclidean_distances(embeddings1[:400], embeddings2[:400])


        labels = np.asarray(self.labels[:400])

        file_output_data = [epoch, steps]

        main_score = None
        for name, scores, reverse in [['Cosine-Similarity', cosine_scores, True], ['Manhatten-Distance', manhattan_distances, False], ['Euclidean-Distance', euclidean_distances, False]]:
            acc, acc_threshold = self.find_best_acc_and_threshold(scores, labels, reverse)
            f1, precision, recall, f1_threshold = self.find_best_f1_and_threshold(scores, labels, reverse)
            ap = average_precision_score(labels, scores * (1 if reverse else -1))

            logger.info("Accuracy with {}:           {:.2f}\t(Threshold: {:.4f})".format(name, acc * 100, acc_threshold))
            logger.info("F1 with {}:                 {:.2f}\t(Threshold: {:.4f})".format(name, f1 * 100, f1_threshold))
            logger.info("Precision with {}:          {:.2f}".format(name, precision * 100))
            logger.info("Recall with {}:             {:.2f}".format(name, recall * 100))
            logger.info("Average Precision with {}:  {:.2f}\n".format(name, ap * 100))

            file_output_data.extend([acc, acc_threshold, f1, precision, recall, f1_threshold, ap])

            if main_score is None: #Use AveragePrecision with Cosine-Similarity as main score
                main_score = ap

        if output_path is not None and self.write_csv:
            csv_path = os.path.join(output_path, self.csv_file)
            if not os.path.isfile(csv_path):
                with open(csv_path, mode="w", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(self.csv_headers)
                    writer.writerow(file_output_data)
            else:
                with open(csv_path, mode="a", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(file_output_data)

        return main_score

    @staticmethod
    def find_best_acc_and_threshold(scores, labels, high_score_more_similar: bool):
        # assert len(scores) == len(labels)
        rows = list(zip(scores, labels))

        rows = sorted(rows, key=lambda x: x[0], reverse=high_score_more_similar)

        max_acc = 0
        best_threshold = -1

        positive_so_far = 0
        remaining_negatives = sum(labels == 0)

        for i in range(len(rows)-1):
            score, label = rows[i]
            if label == 1:
                positive_so_far += 1
            else:
                remaining_negatives -= 1

            acc = (positive_so_far + remaining_negatives) / len(labels)
            if acc > max_acc:
                max_acc = acc
                best_threshold = (rows[i][0] + rows[i+1][0]) / 2

        return max_acc, best_threshold

    @staticmethod
    def find_best_f1_and_threshold(scores, labels, high_score_more_similar: bool):
        # assert len(scores) == len(labels)

        scores = np.asarray(scores)
        labels = np.asarray(labels)

        rows = list(zip(scores, labels))

        rows = sorted(rows, key=lambda x: x[0], reverse=high_score_more_similar)

        best_f1 = best_precision = best_recall = 0
        threshold = 0
        nextract = 0
        ncorrect = 0
        total_num_duplicates = sum(labels)

        for i in range(len(rows)-1):
            score, label = rows[i]
            nextract += 1

            if label == 1:
                ncorrect += 1

            if ncorrect > 0:
                precision = ncorrect / nextract
                recall = ncorrect / total_num_duplicates
                f1 = 2 * precision * recall / (precision + recall)
                if f1 > best_f1:
                    best_f1 = f1
                    best_precision = precision
                    best_recall = recall
                    threshold = (rows[i][0] + rows[i + 1][0]) / 2

        return best_f1, best_precision, best_recall, threshold

### Evaluation

In [ ]:
dev_BCEvaluator = BinaryClassificationEvaluator(dev_dataset,batch_size=BATCH_SIZE,show_progress_bar=True)

In [ ]:
dev_BCEvaluator(model,output_path=folder+'/dev')

Completed 30 iterations
Completed 60 iterations
Completed 90 iterations
Completed 120 iterations
Completed 150 iterations
Completed 180 iterations
Completed 210 iterations
Completed 240 iterations
Completed 270 iterations
Completed 300 iterations
Completed 330 iterations
Completed 360 iterations
Completed 390 iterations
Completed 420 iterations
Completed 450 iterations
Completed 480 iterations
Completed 510 iterations
Completed 540 iterations
Completed 570 iterations
Completed 600 iterations
Completed 630 iterations
Completed 660 iterations
Completed 690 iterations
Completed 720 iterations
Completed 750 iterations
Completed 780 iterations
Completed 810 iterations
Completed 840 iterations
Completed 870 iterations
Completed 900 iterations
Completed 930 iterations
Completed 960 iterations
Completed 990 iterations
Completed 1020 iterations
Completed 1050 iterations
Completed 1080 iterations
Completed 1110 iterations
Completed 1140 iterations
Completed 1170 iterations
Completed 1200 iterati

1.0

## Information retreival evaluator

In [ ]:
import torch
import logging
from tqdm import tqdm, trange
import os
import numpy as np
from typing import List, Tuple, Dict, Set

In [ ]:
class infodataset(Dataset):
    def __init__(self,qr,qr_idx,img_dir,transform = None):
        self.qr = qr
        self.qr_idx = qr_idx
        self.img_dir = img_dir
        self.transform = transform

    def image_adder(self,id1):
        img_id1 = list()
        if((self.qr.at[id1,'Attachments'])!=None):
            for i in self.qr.at[id1,'Attachments']:
                try:
                    img_path = os.path.join(self.img_dir,i)
                    img = Image.open(img_path).convert('RGB')
                    if(self.transform):
                        img = self.transform(img)
                        img.reshape(3,224,224)
                    img_id1.append(img)
                except Exception as e: 
                    print(e)
        else:
            img_id1.append(torch.zeros(3,224,224))

        # Work on this, for few examples, it is still saying list index out of range
        if(len(img_id1)==0):
            # print('No attachments found for id {}'.format(id1))
            print(f'Something went wrong with the image of id {id1}')
            img_id1.append(torch.zeros(3,224,224))

        return img_id1
    
    def __getitem__(self,idx):
        id1 = self.qr_idx[idx]
        img_id1 = self.image_adder(id1)

        # print(len(img_id1))

        # print('Printing id1 {} and len {} and id2 {} and len {} '.format(
        #     id1,len(img_id1),
        #     id2, len(img_id2)
        # ))

        # print('Printing id1 shape {} and id2  shape {}'.format(
        #     img_id1[0].shape,
        #     img_id2[0].shape
        # ))        

        sample = {
            'image': img_id1[0]    #Currently taking only one input image
        }

        t1 = '[CLS]' + self.qr.loc[id1,'Title'] + ' ' + ' '.join(self.qr.loc[id1,'Tags']) + ' ' + self.qr.loc[id1,'Text'] + '[SEP]'
        tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
        t1_token = tokenizer.tokenize(t1)
        indexed_t1 = tokenizer.convert_tokens_to_ids(t1_token)
        
        while(len(indexed_t1)<512):
            indexed_t1.append(0)
        
        ten_t1 = torch.as_tensor(indexed_t1)[:512]
        
        try:
            sample["token"] = ten_t1 # torch.Size([batch_size, 512])
        except Exception as e:
            print(e)
        
        return sample

    def __len__(self):
        return len(self.qr_idx)

In [ ]:
class InformationRetreivalEvaluator():
    def __init__(self,
                 qr,
                 queries: Dict[str, str],  #qid => query
                 corpus: Dict[str, str],  #cid => doc
                 relevant_docs: Dict[str, Set[str]],  #qid => Set[cid]
                 corpus_chunk_size: int = 50000,
                 mrr_at_k: List[int] = [10],
                 ndcg_at_k: List[int] = [10],
                 accuracy_at_k: List[int] = [1, 3, 5, 10],
                 precision_recall_at_k: List[int] = [1, 3, 5, 10],
                 map_at_k: List[int] = [100],
                 show_progress_bar: bool = False,
                 batch_size: int = 32,
                 name: str = '',
                 write_csv: bool = True
                 ):
        
        self.qr = qr
        self.queries_ids = []
        for qid in queries:
            if qid in relevant_docs and len(relevant_docs[qid]) > 0:
                self.queries_ids.append(qid)

        self.queries = [queries[qid] for qid in self.queries_ids]

        self.corpus_ids = list(corpus.keys())
        self.corpus = [corpus[cid] for cid in self.corpus_ids]

        self.relevant_docs = relevant_docs
        self.corpus_chunk_size = corpus_chunk_size
        self.mrr_at_k = mrr_at_k
        self.ndcg_at_k = ndcg_at_k
        self.accuracy_at_k = accuracy_at_k
        self.precision_recall_at_k = precision_recall_at_k
        self.map_at_k = map_at_k

        self.show_progress_bar = show_progress_bar
        self.batch_size = batch_size
        self.name = name
        self.write_csv = write_csv

        if name:
            name = "_" + name

        self.csv_file: str = "Information-Retrieval_evaluation" + name + "_results.csv"
        self.csv_headers = ["epoch", "steps"]


        for k in accuracy_at_k:
            self.csv_headers.append("Accuracy@{}".format(k))

        for k in precision_recall_at_k:
            self.csv_headers.append("Precision@{}".format(k))
            self.csv_headers.append("Recall@{}".format(k))

        for k in mrr_at_k:
            self.csv_headers.append("MRR@{}".format(k))

        for k in ndcg_at_k:
            self.csv_headers.append("NDCG@{}".format(k))

        for k in map_at_k:
            self.csv_headers.append("MAP@{}".format(k))
    
    def __call__(self,model : BridgeModel,output_path: str = None,epoch: int = -1, steps: int = -1) ->float:
        if epoch != -1:
            out_txt = " after epoch {}:".format(epoch) if steps == -1 else " in epoch {} after {} steps:".format(epoch, steps)
        else:
            out_txt = ":"

        logger.info("Information Retrieval Evaluation on " + self.name + " dataset" + out_txt)

        max_k = max(max(self.mrr_at_k), max(self.ndcg_at_k), max(self.accuracy_at_k), max(self.precision_recall_at_k), max(self.map_at_k))

        query_embeddings = self.get_embeddings(model,self.qr,self.queries_ids)

        queries_result_list = [[] for _ in range(len(query_embeddings))]

        itr = range(0, len(self.corpus), self.corpus_chunk_size)

        if self.show_progress_bar:
            itr = tqdm(itr, desc='Corpus Chunks')

        #Iterate over chunks of the corpus
        for corpus_start_idx in itr:
            corpus_end_idx = min(corpus_start_idx + self.corpus_chunk_size, len(self.corpus))

            #Encode chunk of corpus
            sub_corpus_embeddings = self.get_embeddings(model,self.qr,self.corpus_ids[corpus_start_idx:corpus_end_idx])

            #Compute cosine similarites
            cos_scores = pytorch_cos_sim(query_embeddings, sub_corpus_embeddings)
            del sub_corpus_embeddings

            #Get top-k values
            cos_scores_top_k_values, cos_scores_top_k_idx = torch.topk(cos_scores, min(max_k, len(cos_scores[0])), dim=1, largest=True, sorted=False)
            cos_scores_top_k_values = cos_scores_top_k_values.cpu().tolist()
            cos_scores_top_k_idx = cos_scores_top_k_idx.cpu().tolist()
            del cos_scores

            for query_itr in range(len(query_embeddings)):
                for sub_corpus_id, score in zip(cos_scores_top_k_idx[query_itr], cos_scores_top_k_values[query_itr]):
                    corpus_id = self.corpus_ids[corpus_start_idx+sub_corpus_id]
                    queries_result_list[query_itr].append({'corpus_id': corpus_id, 'score': score})


        #Compute scores
        scores = self.compute_metrics(queries_result_list)

        #Output
        self.output_scores(scores)


        # logger.info("Queries: {}".format(len(self.queries)))
        # logger.info("Corpus: {}\n".format(len(self.corpus)))

        if output_path is not None and self.write_csv:
            csv_path = os.path.join(output_path, self.csv_file)
            if not os.path.isfile(csv_path):
                fOut = open(csv_path, mode="w", encoding="utf-8")
                fOut.write(",".join(self.csv_headers))
                fOut.write("\n")

            else:
                fOut = open(csv_path, mode="a", encoding="utf-8")

            output_data = [epoch, steps]
            for k in self.accuracy_at_k:
                output_data.append(scores['accuracy@k'][k])

            for k in self.precision_recall_at_k:
                output_data.append(scores['precision@k'][k])
                output_data.append(scores['recall@k'][k])

            for k in self.mrr_at_k:
                output_data.append(scores['mrr@k'][k])

            for k in self.ndcg_at_k:
                output_data.append(scores['ndcg@k'][k])

            for k in self.map_at_k:
                output_data.append(scores['map@k'][k])

            fOut.write(",".join(map(str,output_data)))
            fOut.write("\n")
            fOut.close()

        return scores['map@k'][max(self.map_at_k)]


    def compute_metrics(self, queries_result_list: List[object]):
        # Init score computation values
        num_hits_at_k = {k: 0 for k in self.accuracy_at_k}
        precisions_at_k = {k: [] for k in self.precision_recall_at_k}
        recall_at_k = {k: [] for k in self.precision_recall_at_k}
        MRR = {k: 0 for k in self.mrr_at_k}
        ndcg = {k: [] for k in self.ndcg_at_k}
        AveP_at_k = {k: [] for k in self.map_at_k}

        # Compute scores on results
        for query_itr in range(len(queries_result_list)):
            query_id = self.queries_ids[query_itr]

            # Sort scores
            top_hits = sorted(queries_result_list[query_itr], key=lambda x: x['score'], reverse=True)
            query_relevant_docs = self.relevant_docs[query_id]

            # Accuracy@k - We count the result correct, if at least one relevant doc is accross the top-k documents
            for k_val in self.accuracy_at_k:
                for hit in top_hits[0:k_val]:
                    if hit['corpus_id'] in query_relevant_docs:
                        num_hits_at_k[k_val] += 1
                        break

            # Precision and Recall@k
            for k_val in self.precision_recall_at_k:
                num_correct = 0
                for hit in top_hits[0:k_val]:
                    if hit['corpus_id'] in query_relevant_docs:
                        num_correct += 1

                precisions_at_k[k_val].append(num_correct / k_val)
                recall_at_k[k_val].append(num_correct / len(query_relevant_docs))

            # MRR@k
            for k_val in self.mrr_at_k:
                for rank, hit in enumerate(top_hits[0:k_val]):
                    if hit['corpus_id'] in query_relevant_docs:
                        MRR[k_val] += 1.0 / (rank + 1)
                        break

            # NDCG@k
            for k_val in self.ndcg_at_k:
                predicted_relevance = [1 if top_hit['corpus_id'] in query_relevant_docs else 0 for top_hit in top_hits[0:k_val]]
                true_relevances = [1] * len(query_relevant_docs)

                ndcg_value = self.compute_dcg_at_k(predicted_relevance, k_val) / self.compute_dcg_at_k(true_relevances, k_val)
                ndcg[k_val].append(ndcg_value)

            # MAP@k
            for k_val in self.map_at_k:
                num_correct = 0
                sum_precisions = 0

                for rank, hit in enumerate(top_hits[0:k_val]):
                    if hit['corpus_id'] in query_relevant_docs:
                        num_correct += 1
                        sum_precisions += num_correct / (rank + 1)

                avg_precision = sum_precisions / min(k_val, len(query_relevant_docs))
                AveP_at_k[k_val].append(avg_precision)

        # Compute averages
        for k in num_hits_at_k:
            num_hits_at_k[k] /= len(self.queries_ids)

        for k in precisions_at_k:
            precisions_at_k[k] = np.mean(precisions_at_k[k])

        for k in recall_at_k:
            recall_at_k[k] = np.mean(recall_at_k[k])

        for k in ndcg:
            ndcg[k] = np.mean(ndcg[k])

        for k in MRR:
            MRR[k] /= len(self.queries_ids)

        for k in AveP_at_k:
            AveP_at_k[k] = np.mean(AveP_at_k[k])


        return {'accuracy@k': num_hits_at_k, 'precision@k': precisions_at_k, 'recall@k': recall_at_k, 'ndcg@k': ndcg, 'mrr@k': MRR, 'map@k': AveP_at_k}


    def output_scores(self, scores):
        for k in scores['accuracy@k']:
            logger.info("Accuracy@{}: {:.2f}%".format(k, scores['accuracy@k'][k]*100))

        for k in scores['precision@k']:
            logger.info("Precision@{}: {:.2f}%".format(k, scores['precision@k'][k]*100))

        for k in scores['recall@k']:
            logger.info("Recall@{}: {:.2f}%".format(k, scores['recall@k'][k]*100))

        for k in scores['mrr@k']:
            logger.info("MRR@{}: {:.4f}".format(k, scores['mrr@k'][k]))

        for k in scores['ndcg@k']:
            logger.info("NDCG@{}: {:.4f}".format(k, scores['ndcg@k'][k]))

        for k in scores['map@k']:
            logger.info("MAP@{}: {:.4f}".format(k, scores['map@k'][k]))


    @staticmethod
    def compute_dcg_at_k(relevances, k):
        dcg = 0
        for i in range(min(len(relevances), k)):
            dcg += relevances[i] / np.log2(i + 2)  #+2 as we start our idx at 0
        return dcg
    
    def get_embeddings(self,model,qr,qr_idx):
        info_dataset = infodataset(qr,qr_idx,img_dir,transform = transform_pipe)
        info_loader = DataLoader(
            info_dataset,
            batch_size=BATCH_SIZE,
            pin_memory=True,
            num_workers = 8,
            )
        
        embeddings = list()
        since = time.time()
        for i,batch in enumerate(info_loader):
            model.eval()

            text = batch['token']
            images = batch['image']

            text,images = torch.as_tensor(text).to(device), torch.as_tensor(images).to(device)

            with torch.no_grad():
                yhat = model.forward(images,text)
            
            for j in yhat:
                embeddings.append(j.cpu().detach().numpy())
            
            if(i%40==0):
                print(f'{i} iterations hase been completed, and model is running for {time.time()-since}')
        
        return embeddings


In [ ]:
with open(folder_quora+'/devinfo_100.txt','rb') as a:
    queries_dev = pickle.load(a)
    rel_docs_dev= pickle.load(a)

with open(folder_quora+'/testinfo_100.txt','rb') as a:
    queries_test= pickle.load(a)
    rel_docs_test= pickle.load(a)

with open(folder_quora+'/traininfo_100.txt','rb') as a:
    queries_train= pickle.load(a)
    rel_docs_train= pickle.load(a)

In [ ]:
def string_sentence(i,qr):
    return qr.loc[i,'Title']+ ' '+' '.join(qr.loc[i,'Tags'])+ ' ' + qr.loc[i,'Text']

In [ ]:
def corpus(qr):
    corpus = dict()
    for i in qr.index.values:
        corpus[i] = string_sentence(i,qr)
    return corpus

In [ ]:
train_inforet = InformationRetreivalEvaluator(train_qr,queries_train,corpus(train_qr),rel_docs_train)
os.makedirs(folder+'/info',exist_ok=True)
train_inforet(model,folder+'/info')

0 iterations hase been completed, and model is running for 0.8856837749481201
40 iterations hase been completed, and model is running for 3.478123188018799


Token indices sequence length is longer than the specified maximum sequence length for this model (802 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1843 > 512). Running this sequence through the model will result in indexing errors


80 iterations hase been completed, and model is running for 6.241954565048218
0 iterations hase been completed, and model is running for 0.9674324989318848
40 iterations hase been completed, and model is running for 3.6424734592437744


Token indices sequence length is longer than the specified maximum sequence length for this model (893 > 512). Running this sequence through the model will result in indexing errors


80 iterations hase been completed, and model is running for 6.236653089523315
120 iterations hase been completed, and model is running for 8.907212257385254
160 iterations hase been completed, and model is running for 11.506620645523071


...

Token indices sequence length is longer than the specified maximum sequence length for this model (1266 > 512). Running this sequence through the model will result in indexing errors


16400 iterations hase been completed, and model is running for 1109.1844940185547


Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


16440 iterations hase been completed, and model is running for 1111.9995603561401


0.00011871156652301434

# Rough